# Capstone Project - The Battle of the Neighborhoods (Week 4)

## Best neighborhood to operate an independant coffee shop in Toronto

### Introduction

This is a "Capstone project" for the final course in the IBM Data Science Professional Certificate, Weeks 4/5. This project aims to determine the best location to open an independent coffee shop using Python scripts, visualization libraries, pandas & numpy packages, FourSquare API, etc.

This project aims to enable aspiring enterpreneures in making an informed choice in terms of location identification by taking into account various factors like population demographics, existing competitors and so on.

### Description and Background of the Problem

For this project, Toronto has been chosen as it has a growing independent coffee shop scene. According to Wikipedia - "Toronto is the capital city of the Canadian province of Ontario. With a recorded population of 2,731,571 in 2016, it is the most populous city in Canada and the fourth most populous city in North America. The city is the anchor of the Golden Horseshoe, an urban agglomeration of 9,245,438 people (as of 2016) surrounding the western end of Lake Ontario, while the Greater Toronto Area (GTA) proper had a 2016 population of 6,417,516. Toronto is an international centre of business, finance, arts, and culture, and is recognized as one of the most multicultural and cosmopolitan cities in the world."

As it is known, choosing a neighborhood and location for a coffee shop is an important and diffucult step for the entrepreneurs. Most of time it can be as crucial factor as it determines the coffee shop's success and also affects the type of service, quality, menu, interior design.

### Description of Data

To identify the best location, the following data is required:

1. The list of Toronto neighborhoods and their boroughs ('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

2. The postal codes of the neighborhoods ('https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')

3. The most updated record of traffic signal vehicle and pedestrian volumes in Toronto.(https://tinyurl.com/vehicle-foot-traffic)

4. The popular or venues of a given neighborhood in Toronto.(https://developer.foursquare.com/)

The List of neighborhood data will be used to obtain the exact coordinates for each neighborhood based on the postal code, this allows exploration of the city map.
Then the coordinates and Foursquare credentials will be used to access the popular venues along with their details, especially for coffee shops. The pedestrian volumes can be used to analyze the foot traffic.

### Pre-Processing the raw data

In [7]:
# importing libraries and packages
import pandas as pd
import numpy as np
import seaborn as sns
import requests
import matplotlib.pyplot as plt
%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

### Neighbourhood Data Analysis

In [8]:
url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup = BeautifulSoup(source)

table_data = soup.find('div', class_='mw-parser-output')
table = table_data.table.tbody

columns = ['PostalCode', 'Borough', 'Neighbourhood']
data = dict({key:[]*len(columns) for key in columns})

for row in table.find_all('tr'):
    for i,column in zip(row.find_all('td'),columns):
        i = i.text
        i = i.replace('\n', '')
        data[column].append(i)

df = pd.DataFrame.from_dict(data=data)[columns]
print(df.shape)
df.head()

(180, 3)


,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
# cleaning the data

df = df[df['Borough'] != 'Not assigned'].reset_index(drop = True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
p, b,n = [],[],[]
for postcode, borough, neigh in zip(df['PostalCode'], df['Borough'], df['Neighbourhood']):
    p.append(postcode)
    b.append(borough)
    if neigh == 'Not assigned':
        n.append(borough)
    else:
        n.append(neigh)
        
df = pd.DataFrame({'PostalCode': p, 'Borough': b, 'Neighbourhood':n})[columns]
print(df.shape)
df.head()

(103, 3)


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [12]:
postcodes = df['PostalCode'].values
boroughs = df['Borough'].values
neighs = df['Neighbourhood'].values

#create a dictionary with keys as Postcode and Borough, keys of dictioaries are unique
dic = dict({(key1,key2): [] for key1, key2 in zip(postcodes, boroughs)})
print('Number of keys in the dictionary are: ', len(dic.keys()))

#filling the values of keys of dictionary
for postcode, borough, neigh in zip(postcodes,boroughs, neighs):
    key = (postcode, borough)
    dic[key].append(neigh)
    
df = pd.DataFrame(columns = ['Postal Code', 'Borough', 'Neighbourhood'])
for key, value in dic.items():
    postcode, borough, neig = key[0], key[1], value
    neig = ', '.join(neig)
    df = df.append({'Postal Code': postcode,
                     'Borough': borough,
                     'Neighbourhood': neig}, ignore_index = True)
print('Shape of final data is: ', df.shape)
df.head()

Number of keys in the dictionary are:  103
Shape of final data is:  (103, 3)


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Getting the Coordinates

In [13]:
url='https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
df_geo=pd.read_csv(url)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df_ll = pd.merge(df, df_geo, on = 'Postal Code')
df_ll.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [15]:
df_ll.shape

(103, 5)

In [16]:
# saving the DataFrame as a new csv file
df_ll.to_csv('toronto_postalcodes_lalo.csv', index = False)
df_ll.shape

(103, 5)